In [3]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam

ModuleNotFoundError: No module named 'tensorflow'

In [5]:
# Étape 1 : Charger le modèle VGG16 pré-entraîné sur ImageNet
base_model = VGG16(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

NameError: name 'VGG16' is not defined

In [7]:
# Étape 2 : Geler les premières couches du modèle
for layer in base_model.layers:
    layer.trainable = False

NameError: name 'base_model' is not defined

In [33]:
# Étape 3 : Ajouter des couches pour adapter à la nouvelle tâche
model = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(5, activation='softmax')  # 5 classes : daisy, dandelion, rose, sunflower, tulip
])

In [35]:
# Compiler le modèle avec des couches gelées
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [37]:
data_dir = "C:/flowers"

In [39]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 20% des données pour la validation
) 

# Générateurs pour l'entraînement et la validation
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 3457 images belonging to 5 classes.
Found 860 images belonging to 5 classes.


In [41]:
# Étape 5 : Entraîner uniquement les nouvelles couches
print("Étape 5 : Entraîner les nouvelles couches")
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_steps=val_generator.samples // val_generator.batch_size
)

Étape 5 : Entraîner les nouvelles couches
Epoch 1/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 453s 4s/step - accuracy: 0.4076 - loss: 2.4833 - val_accuracy: 0.7055 - val_loss: 0.8418
Epoch 2/10
  1/108 ━━━━━━━━━━━━━━━━━━━━ 5:28 3s/step - accuracy: 0.6250 - loss: 0.8977

C:\Users\yasmi\anaconda3\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


108/108 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - accuracy: 0.6250 - loss: 0.8977 - val_accuracy: 0.6786 - val_loss: 0.9410
Epoch 3/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 444s 4s/step - accuracy: 0.6021 - loss: 1.0060 - val_accuracy: 0.7260 - val_loss: 0.7139
Epoch 4/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 6s 28ms/step - accuracy: 0.5938 - loss: 0.9436 - val_accuracy: 0.6786 - val_loss: 0.7130
Epoch 5/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 500s 5s/step - accuracy: 0.6524 - loss: 0.8810 - val_accuracy: 0.7368 - val_loss: 0.7317
Epoch 6/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 8s 33ms/step - accuracy: 0.4688 - loss: 0.9737 - val_accuracy: 0.7143 - val_loss: 0.6836
Epoch 7/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 555s 5s/step - accuracy: 0.6628 - loss: 0.8492 - val_accuracy: 0.7644 - val_loss: 0.7158
Epoch 8/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 6s 24ms/step - accuracy: 0.7500 - loss: 0.6867 - val_accuracy: 0.7500 - val_loss: 0.6555
Epoch 9/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 429s 4s/step - accuracy: 0.6574 - loss: 0.8410 - val_accuracy: 0.760

In [43]:
# Étape 6 : Fine-tuning des couches gelées
print("Étape 6 : Fine-tuning des couches pré-entraînées")
for layer in base_model.layers:
    layer.trainable = True  # Débloquer les couches pour le fine-tuning

Étape 6 : Fine-tuning des couches pré-entraînées


In [45]:
# Recompiler avec un learning rate plus faible
model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [47]:
history_fine_tuning = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_steps=val_generator.samples // val_generator.batch_size
)

Epoch 1/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 3062s 28s/step - accuracy: 0.7028 - loss: 0.7387 - val_accuracy: 0.8377 - val_loss: 0.4690
Epoch 2/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 17s 27ms/step - accuracy: 0.8438 - loss: 0.4016 - val_accuracy: 0.8214 - val_loss: 0.5463
Epoch 3/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 1585s 15s/step - accuracy: 0.7455 - loss: 0.6093 - val_accuracy: 0.8462 - val_loss: 0.4279
Epoch 4/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 16s 27ms/step - accuracy: 0.8125 - loss: 0.4411 - val_accuracy: 0.8571 - val_loss: 0.4263
Epoch 5/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 1607s 15s/step - accuracy: 0.8021 - loss: 0.5012 - val_accuracy: 0.8510 - val_loss: 0.4036
Epoch 6/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 18s 29ms/step - accuracy: 0.8438 - loss: 0.4787 - val_accuracy: 0.8571 - val_loss: 0.4431
Epoch 7/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 1597s 15s/step - accuracy: 0.8256 - loss: 0.4499 - val_accuracy: 0.8558 - val_loss: 0.3863
Epoch 8/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 18s 29ms/step - accuracy: 0.7812 - loss: 0.610